In [2]:
!export LC_ALL=en_US.UTF-8
import tensorflow as tf
import json
from pprint import pprint
import re
import numpy as np
from embedding import *
import nltk

In [3]:
data = json.load(open('../train-v1.1.json'))

In [4]:
from nltk.corpus import stopwords
#nltk.download('stopwords')
nltkStopWords = stopwords.words('english')

In [5]:
def extractor(data):
    contexts = []
    qas = []
    for i in range(len(data["data"])):
        for j in range(len(data["data"][i]["paragraphs"])):
            contexts.append(data["data"][i]["paragraphs"][j]["context"])
            qas.append(data["data"][i]["paragraphs"][j]["qas"])
    return (contexts,qas)

In [6]:
CapPassage = False

from nltk.tokenize import word_tokenize
contexts,qas = extractor(data)

def find_sub_list(sl,l):
    sll=len(sl)
    for ind in (i for i,e in enumerate(l) if e==sl[0]):
        if l[ind:ind+sll]==sl:
            return ind,ind+sll
    return (-1,-1)

def capPassage(passage,answer,cap_length = 30):
    y = np.zeros(cap_length)
    left,right = find_sub_list(answer,passage)
    if(left==-1):
        return passage[0:cap_length]
    left = left - int((cap_length - len(answer))/2)
    right = right + int((cap_length + len(answer))/2)
    if(left < 0):
        left = 0
    if(right > len(passage)):
        right = len(passage)
    return passage[left:right]
    
def findAnsVec(answer,passage):
    ans = np.zeros((len(passage)))
    start,end = find_sub_list(answer,passage)
    if(start==-1):
        start = passage.index(answer[0])
        end = start + len(answer)
    ans[start:end] = 1
    return ans

In [7]:
X_train_comp_all = []
X_train_comp_ans_all = []
X_train_ans_all = []
Y_train_ques_all = []
invalid = 0
X_train_ans_label_all = []
for i,context in enumerate(contexts):
    passage = word_tokenize(context.lower())
    
    a_lab = np.zeros(len(passage))
    for j,_ in enumerate(qas[i]):
        answer = word_tokenize(qas[i][j]["answers"][0]['text'].lower())
        start,end = find_sub_list(answer,passage)
        if start == -1:
            continue
        a_lab[start:end+1] = 1
            
            
    for j,_ in enumerate(qas[i]):
        try:
            question = word_tokenize(qas[i][j]['question'].lower())
            answer = word_tokenize(qas[i][j]["answers"][0]['text'].lower())
            
            if CapPassage:
                cappedPassage = capPassage(passage,answer)
            else:
                cappedPassage = passage
            
            X_train_comp_ans_all.append(findAnsVec(answer,passage))
            X_train_ans_label_all.append(a_lab)
            X_train_comp_all.append(cappedPassage)
            X_train_ans_all.append(answer)
            Y_train_ques_all.append(question)
        except Exception as e:
            invalid = invalid+1
    

In [8]:
from collections import Counter
import operator
def findKMostFrequentWords(k):
    ctr = Counter([item for sublist in X_train_comp_all for item in sublist] + [item for sublist in Y_train_ques_all for item in sublist])
    sorted_ctr = sorted(ctr.items(), key=operator.itemgetter(1), reverse=True)
    return [item[0] for item in sorted_ctr[0:k]]

In [9]:
wordToTake = 50000
words = findKMostFrequentWords(100000)

In [10]:
_word_to_idx_reduced = {}
_idx_to_word_reduced = []


def _add_word_reduced(word):
    idx = len(_idx_to_word_reduced)
    _word_to_idx_reduced[word] = idx
    _idx_to_word_reduced.append(word)
    return idx


UNKNOWN_TOKEN = _add_word_reduced(UNKNOWN_WORD)
START_TOKEN = _add_word_reduced(START_WORD)
END_TOKEN = _add_word_reduced(END_WORD)




dimensions = glove.shape[1]
reduced_glove = []
reduced_glove.append(np.zeros(dimensions))
reduced_glove.append(-np.ones(dimensions))
reduced_glove.append(np.ones(dimensions))

for word in words:
    l = look_up_word(word)
    if(l != UNKNOWN_TOKEN):
        idx = _add_word_reduced(word)
        reduced_glove.append(glove[l])
        if(len(reduced_glove) == wordToTake):
            break
        
def look_up_word_reduced(word):
    return _word_to_idx_reduced.get(word, UNKNOWN_TOKEN)


def look_up_token_reduced(token):
    return _idx_to_word_reduced[token]

reduced_glove = np.array(reduced_glove)
reduced_glove.shape

(50000, 300)

In [11]:
!hostname
print(invalid)
for i in np.where(X_train_ans_label_all[110] == 1)[0]:
    print(X_train_comp_all[110][i])

cuda2.cims.nyu.edu
1510
rev
.
john
j.
cavanaugh
,
c.s.c
.
served
more
than
half
,
lobund
institute
for
animal
studies
and
medieval
institute
.
hall
of
liberal
arts
(


In [12]:
print(X_train_comp_all[0])
print(X_train_ans_all[0])

['architecturally', ',', 'the', 'school', 'has', 'a', 'catholic', 'character', '.', 'atop', 'the', 'main', 'building', "'s", 'gold', 'dome', 'is', 'a', 'golden', 'statue', 'of', 'the', 'virgin', 'mary', '.', 'immediately', 'in', 'front', 'of', 'the', 'main', 'building', 'and', 'facing', 'it', ',', 'is', 'a', 'copper', 'statue', 'of', 'christ', 'with', 'arms', 'upraised', 'with', 'the', 'legend', '``', 'ad', 'me', 'omnes', "''", '.', 'next', 'to', 'the', 'main', 'building', 'is', 'the', 'basilica', 'of', 'the', 'sacred', 'heart', '.', 'immediately', 'behind', 'the', 'basilica', 'is', 'the', 'grotto', ',', 'a', 'marian', 'place', 'of', 'prayer', 'and', 'reflection', '.', 'it', 'is', 'a', 'replica', 'of', 'the', 'grotto', 'at', 'lourdes', ',', 'france', 'where', 'the', 'virgin', 'mary', 'reputedly', 'appeared', 'to', 'saint', 'bernadette', 'soubirous', 'in', '1858.', 'at', 'the', 'end', 'of', 'the', 'main', 'drive', '(', 'and', 'in', 'a', 'direct', 'line', 'that', 'connects', 'through', '

In [13]:
find_sub_list(X_train_ans_all[0] , X_train_comp_all[0])

(101, 104)

In [14]:
print(invalid)
print(X_train_comp_all[101])
print(X_train_ans_all[101])
print(Y_train_ques_all[101])

c = list(zip(X_train_comp_all,X_train_comp_ans_all, X_train_ans_all, X_train_ans_label_all,Y_train_ques_all))
np.random.shuffle(c)
X_train_comp_all_shuffled,X_train_comp_ans_all_shuffled, X_train_ans_shuffled, X_train_ans_label_shuffled,Y_train_ques_all_shuffled = zip(*c)

print(X_train_comp_all_shuffled[101])
print(X_train_comp_ans_all_shuffled[101])
print(X_train_ans_shuffled[101])
print(X_train_ans_label_shuffled[101])
print(Y_train_ques_all_shuffled[101])


1510
['the', 'success', 'of', 'its', 'football', 'team', 'made', 'notre', 'dame', 'a', 'household', 'name', '.', 'the', 'success', 'of', 'note', 'dame', 'reflected', 'rising', 'status', 'of', 'irish', 'americans', 'and', 'catholics', 'in', 'the', '1920s', '.', 'catholics', 'rallied', 'up', 'around', 'the', 'team', 'and', 'listen', 'to', 'the', 'games', 'on', 'the', 'radio', ',', 'especially', 'when', 'it', 'knocked', 'off', 'the', 'schools', 'that', 'symbolized', 'the', 'protestant', 'establishment', 'in', 'america', '—', 'harvard', ',', 'yale', ',', 'princeton', ',', 'and', 'army', '.', 'yet', 'this', 'role', 'as', 'high-profile', 'flagship', 'institution', 'of', 'catholicism', 'made', 'it', 'an', 'easy', 'target', 'of', 'anti-catholicism', '.', 'the', 'most', 'remarkable', 'episode', 'of', 'violence', 'was', 'the', 'clash', 'between', 'notre', 'dame', 'students', 'and', 'the', 'ku', 'klux', 'klan', 'in', '1924.', 'nativism', 'and', 'anti-catholicism', ',', 'especially', 'when', 'dire

In [15]:
examples_to_take_train = 10000

X_train_comp = X_train_comp_all_shuffled[0:examples_to_take_train]
X_train_comp_ans = X_train_comp_ans_all_shuffled[0:examples_to_take_train]
X_train_ans = X_train_ans_shuffled[0:examples_to_take_train]
X_train_ans_label = X_train_ans_label_shuffled[0:examples_to_take_train]
Y_train_ques = Y_train_ques_all_shuffled[0:examples_to_take_train]
answer_indices = [np.where(x==1)[0].tolist() for x in X_train_comp_ans]

In [16]:
max_document_len = len(max(X_train_comp,key=len))
max_answer_len = len(max(X_train_ans,key=len))
max_question_len = len(max(Y_train_ques,key=len)) + 1

In [17]:
document_tokens = np.zeros((examples_to_take_train, max_document_len), dtype=np.int32)
document_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
answer_labels = np.zeros((examples_to_take_train, max_document_len), dtype=np.int32)
answer_masks = np.zeros((examples_to_take_train, max_answer_len, max_document_len), dtype=np.int32)
answer_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
question_input_tokens = np.zeros((examples_to_take_train, max_question_len), dtype=np.int32)
question_output_tokens = np.zeros((examples_to_take_train, max_question_len), dtype=np.int32)
question_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
suppression_answer = np.zeros((examples_to_take_train, reduced_glove.shape[0], 1),dtype=np.int32)
expression_contexts = np.ones((examples_to_take_train, reduced_glove.shape[0], 1),dtype=np.int32)

In [18]:
print(answer_labels.shape)
for i in range(examples_to_take_train):
    answer_labels[i,0:len(X_train_ans_label[i])] = X_train_ans_label[i]
    for j, word in enumerate(X_train_comp[i]):
        document_tokens[i, j] = look_up_word_reduced(word)
    document_lengths[i] = len(X_train_comp[i])

    for j, index in enumerate(answer_indices[i]):
        answer_masks[i, j, index] = 1
    answer_lengths[i] = len(answer_indices[i])
    
    #print(Y_train_ques[i])
    question_input_words = ([START_WORD] + Y_train_ques[i])
    question_output_words = (Y_train_ques[i] + [END_WORD])

    for j, word in enumerate(question_input_words):
            question_input_tokens[i, j] = look_up_word_reduced(word)
    for j, word in enumerate(question_output_words):
        question_output_tokens[i, j] = look_up_word_reduced(word)
    question_lengths[i] = len(question_input_words)
    
    for j, word in enumerate(X_train_ans[i]):
        if(word not in Y_train_ques[i]):
            suppression_answer[i, look_up_word_reduced(word),:] = 1

    for j,word in enumerate(X_train_comp[i]):
        expression_contexts[i,look_up_word_reduced(word),:] = 0

(10000, 766)


In [19]:
def create_vocabulary(data):
    flat_list = [item for sublist in data for item in sublist]
    vocabulary = sorted(set(flat_list))
    vocabulary.append("<UNK>")
    vocabulary.append("unk")
    vocabulary.append("eos")
    vocabulary = ["<EOS>"] + vocabulary
    word_to_index = { word:i for i,word in enumerate(vocabulary) }
    index_to_word = { i:word for i,word in enumerate(vocabulary) }
    return (vocabulary,word_to_index,index_to_word)

In [20]:
print(reduced_glove.shape)
vocabulary_comp,word_to_index_comp,index_to_word_comp = create_vocabulary(X_train_comp + Y_train_ques)
print(len(vocabulary_comp))
print(word_to_index_comp["?"])
print(word_to_index_comp["what"])

(50000, 300)
61672
5881
59463


In [21]:
def create_one_hot_vector(data,vocabulary,word_to_index,index_to_word, maxLen):
    one_hot = np.zeros([maxLen,len(vocabulary)])
    for i,word in enumerate(data):
        if i >= maxLen:
            break
        if(word not in word_to_index):
            word = "<UNK>"
        one_hot[i][word_to_index[word]] = 1
    return one_hot

def create_one_hot_vector_from_indices(data,maxLen,vocabulary):
    one_hot = np.zeros([maxLen,len(vocabulary)])
    for i,indice in enumerate(data):
        if i >= maxLen:
            break
        one_hot[i][int(indice)] = 1
    return one_hot


def create_one_hot_training_Set(data,maxLen,vocabulary):
    one_hot_data = np.zeros([data.shape[0],maxLen,len(vocabulary)])
    for i in range(data.shape[0]):
        one_hot_data[i] = create_one_hot_vector_from_indices(data[i],maxLen,vocabulary)
    return one_hot_data




In [22]:
def sentences_to_indices_glove(X,max_len):
    
    m = len(X)                                 
    
    X_indices = np.full([m,max_len],look_up_word_reduced(END_WORD))
    
    for i in range(m):
        j = 0
        for w in X[i]:
            if(j>=max_len):
                break;
            
            X_indices[i, j] = look_up_word_reduced(w)
            j = j+1
    return X_indices

In [23]:
document_tokens = sentences_to_indices_glove(X_train_comp, max_document_len)

In [24]:
document_tokens[0]

array([   76,    27,   132,  1771,     9,     3,     0,   308,     8,
        1325,   558,     6,   190,   460,    11,   438,     6,     3,
        1125,     5,   438,   172,   138,   558,     4,   265,     7,
        1325,   558,  1043,     3,   818,     5,     3,   604,    13,
         939,  4205,    23,   320,  8234,    15,     0,    14,   532,
        1847,     9,     3,   231,    20,    39,   144,  2261,     5,
         438,     6,    53,  1575,     8,  3965,     4,  4202,     4,
           7,  5223,     6,    36,    11,  5827,     8,     3,     0,
           4,     0,     4,     7,     0,  1068,    44,    29,    31,
          50,    78,   256,  1110,     8,   149,     5,    39,   209,
           6,     3,   155,   460,    11,     3,   393,     5,  4392,
          32, 25436,     6,   101,   294,   214,   100,     3, 11651,
        4392,   432,    10,   367,   393,     8,  5635,     3,  1430,
          18,  5180,   804,   975,     9,     3,   231,     6,  1103,
           4,     3,

In [25]:
answer_labels.shape

(10000, 766)

## Model

In [26]:
cellFlag = 'GRU'
minQuestionLoss = 1.0 

### Document Embedding

In [27]:
import tensorflow.contrib.seq2seq as seq2seq

tf.reset_default_graph()

embedding = tf.get_variable("embedding", initializer=reduced_glove)
embedding = tf.cast(embedding, dtype=tf.float32)
EMBEDDING_DIMENS = glove.shape[1]

d_tokens = tf.placeholder(tf.int32, shape=[None, None], name="d_tokens")
d_lengths = tf.placeholder(tf.int32, shape=[None], name="d_lengths")


document_emb = tf.nn.embedding_lookup(embedding, d_tokens, name="document_emb")
document_emb = tf.cast(document_emb, dtype=tf.float64, name="casted_document_emb")


### Answer Generation

In [28]:
if cellFlag == 'LSTM':
    forward_cell = tf.contrib.rnn.LSTMCell(EMBEDDING_DIMENS)
    backward_cell = tf.contrib.rnn.LSTMCell(EMBEDDING_DIMENS)
elif cellFlag == 'GRU':
    forward_cell = tf.contrib.rnn.GRUCell(EMBEDDING_DIMENS)
    backward_cell = tf.contrib.rnn.GRUCell(EMBEDDING_DIMENS)

answer_outputs, states = tf.nn.bidirectional_dynamic_rnn(
    forward_cell, backward_cell, document_emb, d_lengths, dtype=tf.float64,
    scope="answer_rnn")

answer_outputs = tf.concat(answer_outputs, 2, name="answer_output_concat")

answer_outputs = tf.cast(answer_outputs,tf.float32, name="answer_output_concat")

answer_tags = tf.layers.dense(inputs=answer_outputs, units=2, name="answer_tags")


a_labels = tf.placeholder(tf.int32, shape=[None, None], name="a_labels")

answer_mask = tf.sequence_mask(d_lengths, dtype=tf.float32, name="answer_mask")

answer_loss = seq2seq.sequence_loss(
    logits=answer_tags, targets=a_labels, weights=answer_mask, name="answer_loss")

answer_loss = tf.Print(answer_loss, [answer_loss], message="This is answer_loss: ")

## Question Generation

### Encoder

In [29]:
encoder_input_mask = tf.placeholder(
    tf.float32, shape=[None, None, None], name="encoder_input_mask")
encoder_inputs = tf.matmul(encoder_input_mask, answer_outputs, name="encoder_inputs")
encoder_lengths = tf.placeholder(tf.int32, shape=[None], name="encoder_lengths")

if cellFlag == 'GRU':
    encoder_cell = tf.contrib.rnn.GRUCell(forward_cell.state_size + backward_cell.state_size)
elif cellFlag == 'LSTM':
    encoder_cell = tf.contrib.rnn.LSTMCell(2 * EMBEDDING_DIMENS)



In [30]:
_, encoder_state = tf.nn.dynamic_rnn(
    encoder_cell, encoder_inputs, encoder_lengths, dtype=tf.float32, scope="encoder_rnn")

### Decoder

In [31]:
from tensorflow.python.layers.core import Dense


decoder_inputs = tf.placeholder(tf.int32, shape=[None, None], name="decoder_inputs")
decoder_lengths = tf.placeholder(tf.int32, shape=[None], name="decoder_lengths")

decoder_emb = tf.nn.embedding_lookup(embedding, decoder_inputs,name="decoder_embedding")
decoder_emb = tf.cast(decoder_emb,tf.float32,name="decoder_embedding_cast")

helper = seq2seq.TrainingHelper(decoder_emb , decoder_lengths, name="helper")


projection = Dense(embedding.shape[0], use_bias=False, name="projection")

if cellFlag == 'GRU':
    decoder_cell = tf.contrib.rnn.GRUCell(encoder_cell.state_size)
elif cellFlag == "LSTM":
    decoder_cell = tf.contrib.rnn.LSTMCell(2 * EMBEDDING_DIMENS)

In [32]:
decoder = seq2seq.BasicDecoder(decoder_cell, helper, encoder_state, output_layer=projection)
decoder_outputs, _, _ = seq2seq.dynamic_decode(decoder, scope="decoder")
decoder_outputs = decoder_outputs.rnn_output

decoder_labels = tf.placeholder(tf.int64, shape=[None, None], name="decoder_labels")
question_mask = tf.sequence_mask(decoder_lengths ,dtype=tf.float32)

## Question Generation Loss

In [33]:
# NLL Loss
question_loss = seq2seq.sequence_loss(
    logits=decoder_outputs, targets=decoder_labels, weights=question_mask,
    name="question_loss")
question_loss = tf.Print(question_loss, [question_loss], message="This is question_loss: ")

#Suppression Loss
s_answer = tf.placeholder(tf.float32, shape=[None,None,None], name="suppression_answer")
lambdaSuppress = 0.5

suppression_loss = lambdaSuppress * tf.reduce_sum(tf.matmul(tf.exp(decoder_outputs), s_answer))
suppression_loss = tf.Print(suppression_loss, [suppression_loss], message="This is suppression_loss: ")


#Expression Loss
e_context = tf.placeholder(tf.float32, shape=[None,None,None], name="expression_answer")
lambdaExpress = 0.1

expression_loss = lambdaExpress * tf.reduce_sum(tf.matmul(tf.exp(decoder_outputs), e_context))
expression_loss = tf.Print(expression_loss, [expression_loss], message="This is expression loss: ")

In [34]:
lossFlags = tf.placeholder(shape = (4), dtype=tf.float32)
x = tf.stack([question_loss,answer_loss,suppression_loss,expression_loss])
loss = tf.reduce_sum(tf.multiply(x, lossFlags))

In [35]:
#document_tokens = np.zeros((examples_to_take_train, max_document_len), dtype=np.int32)
#document_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
#answer_labels = np.zeros((examples_to_take_train, max_document_len), dtype=np.int32)
#answer_masks = np.zeros((examples_to_take_train, max_answer_len, max_document_len), dtype=np.int32)
#answer_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
#question_input_tokens = np.zeros((examples_to_take_train, max_question_len), dtype=np.int32)
#question_output_tokens = np.zeros((examples_to_take_train, max_question_len), dtype=np.int32)
#question_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
#suppression_answer = np.zeros((examples_to_take_train,max_answer_len),dtype=np.int32)

In [36]:
def shuffle_list(*ls):
    l =list(zip(*ls))
    np.random.shuffle(l)
    return zip(*l)

In [37]:
import math
def createBatch(inputs,batch_size,shuffle=False):
    outputs = []
    num_batches = math.ceil(len(inputs[0])/batch_size)
    
    for index,inp in enumerate(inputs):
        start = 0
        output = []
        for i in range(num_batches-1):
            maxD = max(inputs[1][start:start+batch_size])
            maxA = max(inputs[4][start:start+batch_size])
            maxQ = max(inputs[7][start:start+batch_size])
            if index == 0 or index == 2:
                output.append(inp[start:start+batch_size,0:maxD]) 
            elif index==3:
                output.append(inp[start:start+batch_size,0:maxA,0:maxD]) 
            elif index==5 or index==6:
                output.append(inp[start:start+batch_size,0:maxQ])
            elif index == 8: # 
                output.append(inp[start:start+batch_size,:,:]) 
            else: 
                output.append(inp[start:start+batch_size])
            start = start + batch_size
        
        # Remaining training sample i.e. training mod batch_size
        maxD = max(inputs[1][start:])
        maxA = max(inputs[4][start:])
        maxQ = max(inputs[7][start:])
        if index == 0 or index == 2:
            output.append(inp[start:,0:maxD]) 
        elif index==3:
            output.append(inp[start:,0:maxA,0:maxD]) 
        elif index==5 or index==6:
            output.append(inp[start:,0:maxQ]) 
        elif index ==8:
            output.append(inp[start:,:, :]) 
        else: 
            output.append(inp[start:])
        outputs.append(output)
    
    return outputs


In [38]:
batch_size = 32
batch_input = createBatch([document_tokens,document_lengths,answer_labels,answer_masks,answer_lengths,question_input_tokens,question_output_tokens,question_lengths,suppression_answer,expression_contexts]
                    ,batch_size)

In [39]:
print("No of features:",len( batch_input))
print("No of batches:",len( batch_input[0]))

No of features: 10
No of batches: 313


In [40]:
optimizer = tf.train.AdamOptimizer(learning_rate=3e-3).minimize(loss)

saver = tf.train.Saver()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.InteractiveSession(config=config)

session.run(tf.global_variables_initializer())
# session.run(tf.variables_initializer(saved_vars))


In [ ]:

EPOCHS = 40
loss_flag = np.array([1,1,0,0])

for epoch in range(1, EPOCHS + 1):
    batch_loss = 0
    print("Epoch {0}".format(epoch))
    for batchNum in range(len(batch_input[0])):
        print("Batch : ",batchNum)
        t = session.run([optimizer, loss, question_loss, answer_loss, suppression_loss, expression_loss], {
            d_tokens: batch_input[0][batchNum],
            d_lengths: batch_input[1][batchNum],
            a_labels: batch_input[2][batchNum],
            encoder_input_mask: batch_input[3][batchNum],
            encoder_lengths: batch_input[4][batchNum],
            decoder_inputs: batch_input[5][batchNum],
            decoder_labels: batch_input[6][batchNum],
            decoder_lengths: batch_input[7][batchNum],
            s_answer: batch_input[8][batchNum],
            e_context: batch_input[9][batchNum],
            lossFlags : loss_flag,
        })
        print("Loss: {0}".format(t[1]))
        batch_loss += t[2]
    batch_loss /= len(batch_input[0])
    print("Batch - Loss: {0}".format(batch_loss))
    if batch_loss < minQuestionLoss:
        loss_flag = np.array([1,1,1,1])
    if(epoch%5 == 0):
        print("Saving model")
        #saver.save(session, "qgen-model")

Epoch 1
Batch :  0
Loss: 11.697443962097168
Batch :  1
Loss: 11.497557640075684
Batch :  2
Loss: 10.212152481079102
Batch :  3
Loss: 8.543424606323242
Batch :  4
Loss: 8.203423500061035
Batch :  5
Loss: 7.71380615234375
Batch :  6
Loss: 8.082677841186523
Batch :  7
Loss: 8.182862281799316
Batch :  8
Loss: 8.126458168029785
Batch :  9
Loss: 8.502995491027832
Batch :  10
Loss: 8.176431655883789
Batch :  11
Loss: 8.149927139282227
Batch :  12
Loss: 8.13116455078125
Batch :  13
Loss: 7.777310848236084
Batch :  14
Loss: 7.7111406326293945
Batch :  15
Loss: 7.822638988494873
Batch :  16
Loss: 7.539064884185791
Batch :  17
Loss: 7.27225923538208
Batch :  18
Loss: 7.47455358505249
Batch :  19
Loss: 7.253452777862549
Batch :  20
Loss: 7.369457721710205
Batch :  21
Loss: 7.182992458343506
Batch :  22
Loss: 7.137919902801514
Batch :  23
Loss: 7.269352436065674
Batch :  24
Loss: 7.018163204193115
Batch :  25
Loss: 7.210416316986084
Batch :  26
Loss: 7.168123245239258
Batch :  27
Loss: 7.3566088676

Loss: 5.897654056549072
Batch :  225
Loss: 5.496918201446533
Batch :  226
Loss: 5.707451343536377
Batch :  227
Loss: 5.405379295349121
Batch :  228
Loss: 6.04636812210083
Batch :  229
Loss: 5.571234703063965
Batch :  230
Loss: 5.180836200714111
Batch :  231
Loss: 5.627233982086182
Batch :  232
Loss: 5.380681991577148
Batch :  233
Loss: 5.498132705688477
Batch :  234
Loss: 5.387778282165527
Batch :  235
Loss: 5.546021938323975
Batch :  236
Loss: 5.409597873687744
Batch :  237
Loss: 5.592176914215088
Batch :  238
Loss: 5.788273811340332
Batch :  239
Loss: 5.996137619018555
Batch :  240
Loss: 5.498562335968018
Batch :  241
Loss: 5.577475070953369
Batch :  242
Loss: 5.909254550933838
Batch :  243
Loss: 5.778346538543701
Batch :  244
Loss: 5.520327568054199
Batch :  245
Loss: 5.594327926635742
Batch :  246
Loss: 5.549367427825928
Batch :  247
Loss: 5.706071376800537
Batch :  248
Loss: 5.3867597579956055
Batch :  249
Loss: 5.561187267303467
Batch :  250
Loss: 5.372147560119629
Batch :  251
L

Loss: 4.66600227355957
Batch :  136
Loss: 4.527903079986572
Batch :  137
Loss: 4.768033027648926
Batch :  138
Loss: 4.694746971130371
Batch :  139
Loss: 4.284581661224365
Batch :  140
Loss: 4.354745864868164
Batch :  141
Loss: 4.774101257324219
Batch :  142
Loss: 4.704472064971924
Batch :  143
Loss: 4.666135787963867
Batch :  144
Loss: 4.800276756286621
Batch :  145
Loss: 4.565735816955566
Batch :  146
Loss: 4.502488613128662
Batch :  147
Loss: 4.747727870941162
Batch :  148
Loss: 4.6367597579956055
Batch :  149
Loss: 4.75193977355957
Batch :  150
Loss: 4.375037670135498
Batch :  151
Loss: 4.762884140014648
Batch :  152
Loss: 4.567432403564453
Batch :  153
Loss: 4.617840766906738
Batch :  154
Loss: 4.5652971267700195
Batch :  155
Loss: 4.633358001708984
Batch :  156
Loss: 4.609301567077637
Batch :  157
Loss: 4.770990371704102
Batch :  158
Loss: 4.3858723640441895
Batch :  159
Loss: 4.749992847442627
Batch :  160
Loss: 4.617266654968262
Batch :  161
Loss: 4.722578525543213
Batch :  162


Loss: 3.8739068508148193
Batch :  45
Loss: 3.946866512298584
Batch :  46
Loss: 4.132576942443848
Batch :  47
Loss: 3.9089112281799316
Batch :  48
Loss: 4.015048027038574
Batch :  49
Loss: 4.1744208335876465
Batch :  50
Loss: 4.1930832862854
Batch :  51
Loss: 4.0706329345703125
Batch :  52
Loss: 3.9548239707946777
Batch :  53
Loss: 4.120948314666748
Batch :  54
Loss: 4.110620498657227
Batch :  55
Loss: 3.9280152320861816
Batch :  56
Loss: 4.199787139892578
Batch :  57
Loss: 4.088376998901367
Batch :  58
Loss: 4.277540683746338
Batch :  59
Loss: 3.9810328483581543
Batch :  60
Loss: 3.867018699645996
Batch :  61
Loss: 3.96267032623291
Batch :  62
Loss: 4.001058578491211
Batch :  63
Loss: 4.170701503753662
Batch :  64
Loss: 3.9758999347686768
Batch :  65
Loss: 3.9628829956054688
Batch :  66
Loss: 3.9290785789489746
Batch :  67
Loss: 3.7595295906066895
Batch :  68
Loss: 3.862755298614502
Batch :  69
Loss: 3.9240424633026123
Batch :  70
Loss: 3.874692678451538
Batch :  71
Loss: 3.74615645408

Loss: 3.2699408531188965
Batch :  266
Loss: 3.364609718322754
Batch :  267
Loss: 3.4416887760162354
Batch :  268
Loss: 3.2833001613616943
Batch :  269
Loss: 3.4181859493255615
Batch :  270
Loss: 3.4282565116882324
Batch :  271
Loss: 3.586915969848633
Batch :  272
Loss: 3.1034140586853027
Batch :  273
Loss: 3.2600677013397217
Batch :  274
Loss: 3.459303617477417
Batch :  275
Loss: 3.4173264503479004
Batch :  276
Loss: 3.3440089225769043
Batch :  277
Loss: 3.66367244720459
Batch :  278
Loss: 3.525322675704956
Batch :  279
Loss: 3.2980809211730957
Batch :  280
Loss: 3.4135866165161133
Batch :  281
Loss: 3.550558090209961
Batch :  282
Loss: 3.227552890777588
Batch :  283
Loss: 3.549752712249756
Batch :  284
Loss: 3.5365047454833984
Batch :  285
Loss: 3.789991855621338
Batch :  286
Loss: 3.4591214656829834
Batch :  287
Loss: 3.258127212524414
Batch :  288
Loss: 3.34641170501709
Batch :  289
Loss: 3.422328472137451
Batch :  290
Loss: 3.1517109870910645
Batch :  291
Loss: 3.2962398529052734
B

Loss: 3.450854778289795
Batch :  174
Loss: 2.798569440841675
Batch :  175
Loss: 2.9220969676971436
Batch :  176
Loss: 2.9451727867126465
Batch :  177
Loss: 2.7294936180114746
Batch :  178
Loss: 3.0572798252105713
Batch :  179
Loss: 2.9392971992492676
Batch :  180
Loss: 2.7815463542938232
Batch :  181
Loss: 3.043476104736328
Batch :  182
Loss: 3.2509074211120605
Batch :  183
Loss: 3.0705039501190186
Batch :  184
Loss: 3.1211655139923096
Batch :  185
Loss: 3.3264336585998535
Batch :  186
Loss: 2.580925941467285
Batch :  187
Loss: 3.039412498474121
Batch :  188
Loss: 2.609219789505005
Batch :  189
Loss: 2.8068103790283203
Batch :  190
Loss: 2.665544033050537
Batch :  191
Loss: 2.853842258453369
Batch :  192
Loss: 2.996521234512329
Batch :  193
Loss: 3.067275285720825
Batch :  194
Loss: 3.2421956062316895
Batch :  195
Loss: 2.8064417839050293
Batch :  196
Loss: 3.1111721992492676
Batch :  197
Loss: 2.7848730087280273
Batch :  198
Loss: 2.8159687519073486
Batch :  199
Loss: 2.82174444198608

Loss: 2.853553056716919
Batch :  81
Loss: 2.906369209289551
Batch :  82
Loss: 2.776461601257324
Batch :  83
Loss: 2.6281557083129883
Batch :  84
Loss: 2.8036465644836426
Batch :  85
Loss: 2.662114381790161
Batch :  86
Loss: 2.4758005142211914
Batch :  87
Loss: 2.295968532562256
Batch :  88
Loss: 2.4272727966308594
Batch :  89
Loss: 2.522841215133667
Batch :  90
Loss: 2.7213685512542725
Batch :  91
Loss: 2.783456563949585
Batch :  92
Loss: 2.703197717666626
Batch :  93
Loss: 2.8685288429260254
Batch :  94
Loss: 2.8162453174591064
Batch :  95
Loss: 3.00897216796875
Batch :  96
Loss: 2.3876168727874756
Batch :  97
Loss: 2.7135541439056396
Batch :  98
Loss: 3.005922794342041
Batch :  99
Loss: 3.042966365814209
Batch :  100
Loss: 2.60452938079834
Batch :  101
Loss: 2.9495861530303955
Batch :  102
Loss: 2.8894033432006836
Batch :  103
Loss: 2.4884719848632812
Batch :  104
Loss: 2.8802359104156494
Batch :  105
Loss: 2.564326524734497
Batch :  106
Loss: 2.8124074935913086
Batch :  107
Loss: 2.

Loss: 2.2962043285369873
Batch :  300
Loss: 2.411257028579712
Batch :  301
Loss: 2.402548313140869
Batch :  302
Loss: 2.118891716003418
Batch :  303
Loss: 2.453585147857666
Batch :  304
Loss: 2.396054744720459
Batch :  305
Loss: 2.3147644996643066
Batch :  306
Loss: 2.070505380630493
Batch :  307
Loss: 1.9851592779159546
Batch :  308
Loss: 2.1464579105377197
Batch :  309
Loss: 2.454075574874878
Batch :  310
Loss: 2.314377784729004
Batch :  311
Loss: 2.2324864864349365
Batch :  312
Loss: 1.6449772119522095
Batch - Loss: 2.3665644669304258
Saving model
Epoch 6
Batch :  0
Loss: 2.140312671661377
Batch :  1
Loss: 2.5062689781188965
Batch :  2
Loss: 2.217703104019165
Batch :  3
Loss: 1.8754558563232422
Batch :  4
Loss: 2.0199294090270996
Batch :  5
Loss: 2.2777631282806396
Batch :  6
Loss: 2.2527968883514404
Batch :  7
Loss: 2.09658145904541
Batch :  8
Loss: 2.2076005935668945
Batch :  9
Loss: 2.36757230758667
Batch :  10
Loss: 2.407931089401245
Batch :  11
Loss: 2.103924512863159
Batch :  

In [ ]:
saver = tf.train.Saver()
saver.save(session, "qgen-model")

In [ ]:
batch_input[3][0].shape

In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
saver = tf.train.Saver()

saver.restore(session, 'qgen-model')

In [ ]:
answers = session.run(answer_tags, {
    d_tokens: batch_input[0][2],
    d_lengths: batch_input[1][2],
})
answers = np.argmax(answers, 2)

In [ ]:
for i in range(32):
    print("Prediction")
    printAllAns(answers,2,0)
    print("Ground Truth")
    printAllAns(batch_input[2][2],2,0)
answers[0]

In [ ]:
def printDoc(batch,num):
    for i in batch_input[0][batch][num]:
        print(look_up_token_reduced(i),sep=" ", end=" ")
    print(" ")

def printQues(batch,num):
    for i in batch_input[5][batch][num]:
        print(look_up_token_reduced(i),sep=" ", end=" ")
    print(" ")
    
def printAnsForQuestion(batch, num):
    for i in batch_input[5][batch][num]:
        print(look_up_token_reduced(i),sep=" ", end=" ")
    print(" ")
    
def printAllAns(answers, batch, num):
    for i,word in enumerate(batch_input[0][batch][num]):
        if answers[num][i] == 1 :
            print(look_up_token_reduced(word),sep=" ", end=" ")
    print(" ")
    

In [ ]:
import itertools

batchNum = 2

helper = seq2seq.GreedyEmbeddingHelper(embedding, tf.fill([batch_input[0][batchNum].shape[0]], START_TOKEN), END_TOKEN)
decoder = seq2seq.BasicDecoder(decoder_cell, helper, encoder_state, output_layer=projection)
decoder_outputs, _, _ = seq2seq.dynamic_decode(decoder, maximum_iterations=max_question_len)
decoder_outputs = decoder_outputs.rnn_output


questions = session.run(decoder_outputs, {
    d_tokens: batch_input[0][batchNum],
    d_lengths: batch_input[1][batchNum],
    a_labels: batch_input[2][batchNum],
    encoder_input_mask: batch_input[3][batchNum],
    encoder_lengths: batch_input[4][batchNum],
})



In [ ]:
batch_input[0][batchNum].shape

In [ ]:
#questions[:,:,END_TOKEN] = 0
qs = np.argmax(questions, 2)

In [ ]:
# q = set()
l = []
for i in range(batch_input[0][batchNum].shape[0]):
    print("---------------------------------------------------------------------------------------------")
    question = itertools.takewhile(lambda t: t != END_TOKEN, qs[i])
    print("Generated Question: " + " ".join(look_up_token_reduced(token) for token in question))
    print("Ground Truth Question: ")
    printQues(batchNum,i)
    print("Ground Truth Answer: ", X_train_ans_shuffled[batch_size*batchNum + i])
    print("Context:")
    printDoc(batchNum,i)
    print("---------------------------------------------------------------------------------------------")
    
    
# q

In [ ]:
batch_input[5][18].shape

In [ ]:
batch_input[5][1].shape

In [ ]:
batch_input[6][1].shape

In [ ]:
max(batch_input[7][1])

In [ ]:
max_question_len

In [ ]:
len(batch_input)

In [ ]:
reduced_glove